In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

company_names_df = pd.read_json('/content/drive/MyDrive/DSAA5002/A_share_list.json')

news_df = pd.read_excel('/content/drive/MyDrive/DSAA5002/News.xlsx')

Exception ignored in: <function ZipFile.__del__ at 0x7caadc9cd900>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [ ]:
company_names_df

,name,fullname,code,location,time
0,邵阳液压,邵阳维克液压股份有限公司,301079,深圳证券交易所,2021-10-19
1,同益中,北京同益中新材料科技股份有限公司,688722,上海证券交易所,2021-10-19
2,华瓷股份,湖南华联瓷业股份有限公司,001216,深圳证券交易所,2021-10-19
3,鸿富瀚,深圳市鸿富瀚科技股份有限公司,301086,深圳证券交易所,2021-10-20
4,高铁电气,中铁高铁电气装备股份有限公司,688285,上海证券交易所,2021-10-20
...,...,...,...,...,...
4649,众信旅游,众信旅游集团股份有限公司,002707.SZ,深圳证券交易所,2014-01-23
4650,北新建材,北新集团建材股份有限公司,000786.SZ,深圳证券交易所,1997-06-06
4651,乾景园林,北京乾景园林股份有限公司,603778.SH,上海证券交易所,2015-12-31
4652,能科股份,能科科技股份有限公司,603859.SH,上海证券交易所,2016-10-21


In [ ]:
news_df.head()

,NewsID,Title,NewsContent,NewsSource
0,1,建设银行原董事长张恩照一审被判15年,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,中国证券报
1,2,农行信用卡中心搬到上海滩,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,人民日报
2,3,外运发展：价值型蓝筹股补涨要求强烈,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,杭州新希望
3,4,胜利股份：稳步走强形成标准上升通道,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,源达投资
4,5,[港股快讯]恒指收市报18960点 成交467亿港元,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点...,全景网


In [ ]:
news_df['NewsContent'] = news_df['NewsContent'].astype(str)

In [ ]:
from fuzzywuzzy import fuzz

# Create a new column to store the mentioned company names
news_df['Explicit Company'] = ''

# Create an empty DataFrame to store the news that meet the conditions
filtered_news_df = pd.DataFrame(columns=news_df.columns)

# Iterate over each news item
for i, news in news_df.iterrows():
    # Create a list to store the mentioned company names
    mentioned_companies = []
    # Iterate over each company name
    for j, company in company_names_df.iterrows():
        # Calculate the Levenshtein ratio
        ratio = fuzz.ratio(news['NewsContent'], company['name'])
        # If the Levenshtein ratio is greater than 80, add the company name to the list
        if ratio > 80:
            mentioned_companies.append(company['name'])
    # If the list is not empty, add the news to filtered_news_df
    if mentioned_companies:
        news['Explicit Company'] = ', '.join(mentioned_companies)
        filtered_news_df = filtered_news_df.append(news)

In [ ]:
import pandas as pd

company_names_df['processed name'] = company_names_df['name'].str.replace('股份', '')
# Load the list of China A-share listed companies from the company_names_df dataframe
listed_companies = company_names_df['processed name'].tolist()

# Create an empty list to store the filtered news rows
filtered_rows = []

# Iterate over each row in the news dataframe
for index, row in news_df.iterrows():
    news_content = row['NewsContent']
    mentioned_companies = []

    # Check if the news content mentions any of the company names
    for company in listed_companies:
        if company in news_content:
            mentioned_companies.append(company)

    # If any company is mentioned, add the row to the filtered news
    if mentioned_companies:
        row['Explicit_Company'] = ', '.join(mentioned_companies)
        filtered_rows.append(row)

# Create a new dataframe with the filtered news rows
filtered_news_df = pd.DataFrame(filtered_rows)

# Reset the index of the filtered dataframe
filtered_news_df.reset_index(drop=True, inplace=True)

# Print the filtered news dataframe
print(filtered_news_df)

         NewsID                                    Title  \
0             1                       建设银行原董事长张恩照一审被判15年   
1             2                             农行信用卡中心搬到上海滩   
2             3                        外运发展：价值型蓝筹股补涨要求强烈   
3             4                        胜利股份：稳步走强形成标准上升通道   
4             8                      南风化工：钾肥三雄之一 被"中化"相中   
...         ...                                      ...   
462459  1037031    亿华通：公司电解槽相关产品目前还处于产品的研发及测试阶段 尚未实现批量销售   
462460  1037032                             依米康：接受中泰证券调研   
462461  1037033         天风证券给予中核科技买入评级 核电行业景气上行 公司有望乘风而起   
462462  1037034  海特生物：公司在抗癌药CPT获批后 会考虑适时开展CPT在海外的临床并谋求上市   
462463  1037035                      恩捷股份：股东合益投资部分股份补充质押   

                                              NewsContent NewsSource  \
0       　　本报记者 田雨 李京华    　　中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...      中国证券报   
1       　　中国农业银行信用卡中心由北京搬到上海了！  　　农行行长杨明生日前在信用卡中心揭牌仪式上...       人民日报   
2       　　在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...      

In [ ]:
filtered_news_df.to_excel('/content/drive/MyDrive/DSAA5002/filtered_news.xlsx', index=False)

In [ ]:
import pandas as pd

train_data = pd.read_csv('/content/drive/MyDrive/DSAA5002/Train_Data.csv')

In [ ]:
train_data.head()

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Prepare the dataset
sentences = train_data['text'].tolist()  # List of preprocessed sentences
labels = train_data['label'].tolist()  # List of corresponding sentiment labels

# Clear the sentences
sentences = [clear_character(sentence) for sentence in sentences]
sentences = clean_stopwords(sentences)

# Tokenize the sentences and convert them into input sequences
input_ids = []
attention_masks = []

for sentence in sentences:
    encoded_inputs = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids.append(encoded_inputs['input_ids'])
    attention_masks.append(encoded_inputs['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Split the dataset into train and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, val_masks, train_input_ids, val_input_ids = train_test_split(attention_masks, input_ids, test_size=0.2, random_state=42)

# Create data loaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(val_inputs, val_masks, val_labels)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(5):  # Number of training epochs
    model.train()

    for batch in train_loader:
        input_ids, attention_masks, labels = batch
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

    # Evaluation on validation set
    model.eval()
    val_loss, val_accuracy = 0, 0
    total_val_examples = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_masks, labels = batch
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
            val_loss += outputs.loss.item()
            _, predicted_labels = torch.max(outputs.logits, dim=1)
            val_accuracy += (predicted_labels == labels).sum().item()
            total_val_examples += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    avg_val_accuracy = val_accuracy / total_val_examples

    print(f"Epoch {epoch+1}: Val Loss = {avg_val_loss:.4f}, Val Accuracy = {avg_val_accuracy:.4f}")

# Save the trained model
model.save_pretrained('/content/drive/MyDrive/DSAA5002/')
tokenizer.save_pretrained('/content/drive/MyDrive/DSAA5002/')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
import re
#Clear uselss characters for Chinese setiment analysis
def clear_character(sentence):
    pattern1= '\[.*?\]'
    pattern2 = re.compile('[^\u4e00-\u9fa5^a-z^A-Z^0-9]')
    line1=re.sub(pattern1,'',sentence)
    line2=re.sub(pattern2,'',line1)
    new_sentence=''.join(line2.split())
    return new_sentence


In [ ]:
import jieba
#Delete Chinese stop words
def clean_stopwords(contents):
    contents_list=[]
    stopwords = {}.fromkeys([line.rstrip() for line in open('/content/drive/MyDrive/DSAA5002/baidu_stopwords.txt', encoding="utf-8")])
    stopwords_list = set(stopwords)
    for row in contents:
        words_list = jieba.lcut(row)
        words = [w for w in words_list if w not in stopwords_list]
        sentence=''.join(words)
        contents_list.append(sentence)
    return contents_list

In [ ]:
#Directly run from here, previous model has been trained and saved. Excel has also been saved to avoid duplicated run time

import pandas as pd

filtered_news_df = pd.read_excel('/content/drive/MyDrive/DSAA5002/filtered_news.xlsx')

filtered_news_df = filtered_news_df
# Preprocess the dataframe

sentences = filtered_news_df['NewsContent'].tolist()

cleaned_sentences = [clear_character(sentence) for sentence in sentences]
cleaned_sentences = clean_stopwords(cleaned_sentences)


In [ ]:
# We no longer used the pretrain model and we use a trained model instead
#!pip install sentence-transformers

# from transformers import AutoTokenizer, AutoModelForMaskedLM
# from sentence_transformers import SentenceTransformer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# model = SentenceTransformer("bert-base-chinese")

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# 指定预训练模型的路径
model_name = 'hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2'

# 加载预训练模型
model = BertForSequenceClassification.from_pretrained(model_name)

# 加载对应的分词器
tokenizer = BertTokenizer.from_pretrained(model_name)

# Create an empty list to store the predicted sentiment labels
predicted_labels = []
i = 0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 创建设备对象

model = model.to(device)  # 将模型移动到设备上

# Perform sentiment analysis using the BERT model for each sentence
for sentence in cleaned_sentences:
    # Tokenize the sentence
    encoded_inputs = tokenizer(sentence, padding=True, truncation=True, max_length=512, return_tensors='pt')

    # Move the encoded inputs to the device
    encoded_inputs = encoded_inputs.to(device)

    # Perform sentiment analysis using the BERT model
    with torch.no_grad():
        outputs = model(**encoded_inputs)

        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
    i += 1
    # Add the predicted sentiment label to the list
    predicted_labels.append(predicted_label)
    if i % 1000 == 0:
      print(f"Processed {i} rows")
# Add the predicted sentiment labels as a new column to the dataframe

filtered_news_df['label'] = predicted_labels

Processed 1000 rows
Processed 2000 rows
Processed 3000 rows
Processed 4000 rows
Processed 5000 rows
Processed 6000 rows
Processed 7000 rows
Processed 8000 rows
Processed 9000 rows
Processed 10000 rows
Processed 11000 rows
Processed 12000 rows
Processed 13000 rows
Processed 14000 rows
Processed 15000 rows
Processed 16000 rows
Processed 17000 rows
Processed 18000 rows
Processed 19000 rows
Processed 20000 rows
Processed 21000 rows
Processed 22000 rows
Processed 23000 rows
Processed 24000 rows
Processed 25000 rows
Processed 26000 rows
Processed 27000 rows
Processed 28000 rows
Processed 29000 rows
Processed 30000 rows
Processed 31000 rows
Processed 32000 rows
Processed 33000 rows
Processed 34000 rows
Processed 35000 rows
Processed 36000 rows
Processed 37000 rows
Processed 38000 rows
Processed 39000 rows
Processed 40000 rows
Processed 41000 rows
Processed 42000 rows
Processed 43000 rows
Processed 44000 rows
Processed 45000 rows
Processed 46000 rows
Processed 47000 rows
Processed 48000 rows
P

In [ ]:
filtered_news_df['label'] = filtered_news_df['label'].replace({2: 1, 1: 0})

In [ ]:
task_1 = pd.DataFrame()
task_1['NewsID']= filtered_news_df['NewsID']
task_1['NewsContent']= filtered_news_df['NewsContent']
task_1['Explicit_Company']= filtered_news_df['Explicit_Company']
task_1['label'] = filtered_news_df['label']
filtered_news_df.to_excel('/content/drive/MyDrive/DSAA5002/Task_1_Liwei_Ye_50014738.xlsx', index=False)

In [ ]:
import pandas as pd

task_1 = pd.read_excel('/content/drive/MyDrive/DSAA5002/Task_1_Liwei_Ye_50014738.xlsx')

In [ ]:
import pandas as pd

company_list = pd.read_csv('/content/drive/MyDrive/DSAA5002/KnowledgeGraph/hidy.nodes.company.csv')

relation_df_1 = pd.read_csv('/content/drive/MyDrive/DSAA5002/KnowledgeGraph/hidy.relationships.compete.csv')
relation_df_2 = pd.read_csv('/content/drive/MyDrive/DSAA5002/KnowledgeGraph/hidy.relationships.cooperate.csv')
relation_df_3 = pd.read_csv('/content/drive/MyDrive/DSAA5002/KnowledgeGraph/hidy.relationships.dispute.csv')
relation_df_4 = pd.read_csv('/content/drive/MyDrive/DSAA5002/KnowledgeGraph/hidy.relationships.invest.csv')
relation_df_5 = pd.read_csv('/content/drive/MyDrive/DSAA5002/KnowledgeGraph/hidy.relationships.same_industry.csv')
relation_df_6 = pd.read_csv('/content/drive/MyDrive/DSAA5002/KnowledgeGraph/hidy.relationships.supply.csv')

relation_df = pd.concat([relation_df_1, relation_df_2, relation_df_3, relation_df_4, relation_df_5, relation_df_6])


In [ ]:
# 使用merge函数替换startID和endID
relation_df = relation_df.merge(company_list, left_on=':START_ID', right_on=':ID', how='left')
relation_df = relation_df.rename(columns={'company_name': 'startCompanyName'})
relation_df = relation_df.drop(':START_ID', axis=1)

relation_df = relation_df.merge(company_list, left_on=':END_ID', right_on=':ID', how='left')
relation_df = relation_df.rename(columns={'company_name': 'endCompanyName'})
relation_df = relation_df.drop(':END_ID', axis=1)

# 显示更新后的数据框
print(relation_df)

         :TYPE             time  :ID_x startCompanyName     code_x :LABEL_x  \
0      compete   2020/4/9 13:33   2082             中兴通讯  000063.SZ  company   
1      compete  2018/10/29 7:16   3348             中国平安  601318.SH  company   
2      compete   2021/5/17 8:01   1431             分众传媒  002027.SZ  company   
3      compete  2022/1/24 20:25    272             贵州茅台  600519.SH  company   
4      compete  2021/7/21 20:48    743             宁德时代  300750.SZ  company   
...        ...              ...    ...              ...        ...      ...   
11661   supply              NaN   2356             润建股份  002929.SZ  company   
11662   supply              NaN   3176             科远智慧  002380.SZ  company   
11663   supply              NaN   1055             长亮科技  300348.SZ  company   
11664   supply              NaN   3463              飞荣达  300602.SZ  company   
11665   supply              NaN   3863             西部建设  002302.SZ  company   

       :ID_y endCompanyName     code_y :LABEL_y  
0

In [ ]:
relation_df = relation_df[[':TYPE','startCompanyName','endCompanyName']]
# 根据':TYPE'列的值创建新的'sentiment_effect'列
relation_df['sentiment_effect'] = relation_df[':TYPE'].apply(lambda x: 'opposite' if x in ['compete', 'dispute'] else 'same')

# 显示更新后的数据框
print(relation_df)

         :TYPE startCompanyName endCompanyName sentiment_effect
0      compete             中兴通讯           杰瑞股份         opposite
1      compete             中国平安           贵州茅台         opposite
2      compete             分众传媒           中国联通         opposite
3      compete             贵州茅台           宁德时代         opposite
4      compete             宁德时代           贵州茅台         opposite
...        ...              ...            ...              ...
11661   supply             润建股份           宜通世纪             same
11662   supply             科远智慧           江苏国信             same
11663   supply             长亮科技           上海银行             same
11664   supply              飞荣达            比亚迪             same
11665   supply             西部建设           海亮股份             same

[11666 rows x 4 columns]


In [ ]:
def get_implicit_companies(df):
    explicit_companies = df['Explicit_Company'].split(',')
    label = df['label']

    positive_companies = []
    negative_companies = []

    for explicit_company in explicit_companies:
        explicit_company = explicit_company.strip()  # remove leading and trailing spaces

        related_rows = relation_df[(relation_df['startCompanyName'] == explicit_company) | (relation_df['endCompanyName'] == explicit_company)]

        if related_rows.empty:
            continue

        if label == 0:
            positive = list(set(related_rows[related_rows['sentiment_effect'] == 'opposite']['endCompanyName'].tolist())) if 'opposite' in related_rows['sentiment_effect'].values else ['none']
            negative = list(set(related_rows[related_rows['sentiment_effect'] == 'same']['endCompanyName'].tolist())) if 'same' in related_rows['sentiment_effect'].values else ['none']
        else:
            positive = list(set(related_rows[related_rows['sentiment_effect'] == 'same']['endCompanyName'].tolist())) if 'same' in related_rows['sentiment_effect'].values else ['none']
            negative = list(set(related_rows[related_rows['sentiment_effect'] == 'opposite']['endCompanyName'].tolist())) if 'opposite' in related_rows['sentiment_effect'].values else ['none']

        # Ensure the explicit company is not in the implicit companies lists
        if explicit_company in positive:
            positive.remove(explicit_company)
        if explicit_company in negative:
            negative.remove(explicit_company)

        positive_companies.extend(positive)
        negative_companies.extend(negative)

    return pd.Series([list(set(positive_companies)), list(set(negative_companies))])

task_1[['Implicit_Positive_Company', 'Implicit_Negative_Company']] = task_1.apply(get_implicit_companies, axis=1)

# 显示更新后的数据框
print(task_1)

         NewsID                                    Title  \
0             1                       建设银行原董事长张恩照一审被判15年   
1             2                             农行信用卡中心搬到上海滩   
2             3                        外运发展：价值型蓝筹股补涨要求强烈   
3             4                        胜利股份：稳步走强形成标准上升通道   
4             8                      南风化工：钾肥三雄之一 被"中化"相中   
...         ...                                      ...   
462459  1037031    亿华通：公司电解槽相关产品目前还处于产品的研发及测试阶段 尚未实现批量销售   
462460  1037032                             依米康：接受中泰证券调研   
462461  1037033         天风证券给予中核科技买入评级 核电行业景气上行 公司有望乘风而起   
462462  1037034  海特生物：公司在抗癌药CPT获批后 会考虑适时开展CPT在海外的临床并谋求上市   
462463  1037035                      恩捷股份：股东合益投资部分股份补充质押   

                                              NewsContent NewsSource  \
0       　　本报记者 田雨 李京华    　　中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...      中国证券报   
1       　　中国农业银行信用卡中心由北京搬到上海了！  　　农行行长杨明生日前在信用卡中心揭牌仪式上...       人民日报   
2       　　在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...      

In [ ]:
task_1.to_excel('/content/drive/MyDrive/DSAA5002/Task_2_Liwei_Ye_50014738.xlsx', index=False)

In [ ]:
!pip list --format=freeze

absl-py==1.4.0
aiohttp==3.9.1
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.3.1
altair==4.2.2
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.1.0
audioread==3.0.1
autograd==1.6.2
Babel==2.13.1
backcall==0.2.0
beautifulsoup4==4.11.2
bidict==0.22.1
bigframes==0.15.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.1
bqplot==0.12.42
branca==0.7.0
build==1.0.3
CacheControl==0.13.1
cachetools==5.3.2
catalogue==2.0.10
certifi==2023.11.17
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.7
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.27.7
cmdstanpy==1.2.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==41.0.7
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.2
cvxpy==1.3.2
cycler==0.12.1
c